## Código

### Procesamiento del CSV

In [1]:
#include <iostream>
#include <string>
#include <vector>
#include <limits>
#include <xtensor/xarray.hpp>
#include <xtensor/xadapt.hpp>
#include <xtensor/xio.hpp> // Include this header for printing
#include <fstream> // For opening file
#include <xtensor/xcsv.hpp> // For exporting to csv at the end
#include "xtensor/xview.hpp"
#include "rapidcsv.h"

// Dejo una función para covertir fechas con el formato que quiero a segundos desde el epoch
std::vector<std::time_t> convert_dates_to_seconds(const std::vector<std::string>& date_strings) 
{
    std::vector<std::time_t> seconds_since_epoch;

    for (const auto& date_str : date_strings) {
        std::tm tm = {};
        std::istringstream ss(date_str);

        // Parse the date string using the appropriate format
        ss >> std::get_time(&tm, "%m/%d/%Y %H:%M");
        if (ss.fail()) {
            std::cerr << "Failed to parse date: " << date_str << std::endl;
            continue; // Skip this entry on failure
        }

        // Convert to time_t (seconds since epoch)
        std::time_t time = std::mktime(&tm);
        seconds_since_epoch.push_back(time);
    }

        return seconds_since_epoch;
}

using namespace std;
using namespace rapidcsv;

    // LabelParams (a,b), defecto es (0,-1), pero dejarlo así. Con a=0 usa la primer fila de header, los interpreta como string.
    // El b se me hace misterioso dejarlo en -1, pero si le ponemos 0, interpreta TODO como string dps.
    // ConverterParams(true) ES PARA MISSING VALUES (y Wrong): Para datos float les pone el tipo std::numeric_limits<long double>::signaling_NaN(), y para ints es 0 (pero como long long, no int).
    // FACILMENTE PODEMOS DECIDIR NOSOTROS el tipo llamando la funcion así ConverterParams(true, ELTIPOPARAFLOAT, ELTIPOPARAINT, true), yo RECOMIENDO usar NaN de tipo DOUBLE si en nuestro código usamos double para los float, y tipo int para int si es nuestro código usa "ints" (y no long long que es feo). SI DEJAMOS LOS DEFAULT VAMOS A TENER WARNING DE UN "NARROWING" porque acá definimos un longdouble de nan y en el código se usa un double y a mí ME CAGABA por ejemplo los nanamean interpretaba MAL los NAN!, OJO!
    // Ejemplo si los queiro float e ints como NaN pongo ConverterParams (false,std::numeric_limits<long double>::signaling_NaN(),std::numeric_limits<long double>::signaling_NaN(), true)
    // NOTA: Para un int no se considera NaN como tan válido no sé por qué, se usa quizá std::numeric_limits<long double>::max() que es el máximo int representable que es gigantezco.
    // NOTA 2: ES NECESARIO ADEMÁS para realizar conversiones en general depsués estos parámetros así que está bueno tenerlos definidos de UNA
    // NOTA 3: Si queremos los default NO los ponemos, o escribimos rapidcsv:ConverterParams(true) (que es poner el primer parámetro nomás)
    using namespace rapidcsv;
    ConverterParams params(true, numeric_limits<long double>::signaling_NaN(), numeric_limits<long long>::signaling_NaN(), true);
    Document doc("./Exp_Octubre.csv", LabelParams(), SeparatorParams(';'), ConverterParams(params) );
    // Dps de este comando guardó los datos como byte digamos, después los podemos castear a lo que queramos y allí habrá una intepreteación de cómo eran los datos
    // de manera individual o en vectors que es lo más útil.
    // Lo bueno es que lo podemos castear a lo que queramos básicamente.

    // Removemos las columnas que no usaremos en absoluto
    doc.RemoveColumn("description");
    doc.RemoveColumn("kind");
    
    // Extract the columns as strings
    // En los nombres de las variables nos alcanza con dejarlo como auto.
    // PERO, en GetColumn podemos CASTEAR (entre brackets), interpretó un tipo pero si era int nos los pasa a float, o lo dejamos como int is queremos
    // EPA está bueno

    // Creamos un vector que contenga todas las columnas (casteadas como string, las vamos a modificar)

    vector<vector<string>> numeric_columns_str =
    {
        doc.GetColumn<string>("bid"), // COL 0 
        doc.GetColumn<string>("ask"), // COL 1
        doc.GetColumn<string>("underBid"), // COL 2
        doc.GetColumn<string>("underAsk"), // COL 3
    };

    // Guardamos la columna de fechas de manera separada
    auto created_at = doc.GetColumn<string>("created_at");
    // Convertimos a segundos con la función ya definida para ser manipulado adecuadamente con xtensor
    // auto created_at_seconds = convert_dates_to_seconds(created_at);

    // Hay que realizar las siguientes modificaciones
    // - reemplazar comas por puntos praa correcta interpretacion
    // - convertir de strin a double teniendo en cuenta valroes incorrectos como nan

    // CREAMOS una nueva lista-vector donde colocaremos las filas con las modificaciones necesarias
    vector< vector<double> > numeric_columns;

    // Realizamos las modificaciones a las columnas anteriores y las agregamos a este vector final

    for (auto column : numeric_columns_str)
    {
        // Creamos una columna temporal a modificar
        vector<double> modified_column;

        // Modificamos dentro de los items de cada columna
        for (auto value : column)
        {
            // Reemplazamos las comas por puntos
            replace(value.begin(), value.end(), ',', '.'); // Replace commas with points
            double tempdouble;
            // Convertimos de string a double, usamos rapidcsv mismo que maneja datos incorrectos automaticamente
            // (o sea rapidcsv::Converter)
            // (alternativa con un if sería std::stof)
            Converter<double>(params).ToVal(value, tempdouble );
            //
            modified_column.push_back( tempdouble );
        }
        // Colocamos la columna modificada en la lista-vector de columnas finales
        numeric_columns.push_back(modified_column);
    }

    // for (auto value: numeric_columns[3])
    // {
    //     cout << value << endl;
    // }

Funciones necesarias para los cálculos

In [2]:
// Define constants, so we calculate sqrt only once in the code
const double sqrt_0_5 = sqrt(0.5);

In [3]:
// Cumulative distribution function as function of native complementary error function in c++
double normalCDF(double z_value) {
    return 0.5 * erfc( - (z_value * sqrt_0_5) );
    // return 0.5 * (1 + erf(z_value * sqrt_0_5) ) ;
    // return 0.5 * erfc( (z_value / sqrt_0_5) );
}

In [4]:
// Define the function to optimize (Black-Scholes formula)
double function_to_optimize(double S, double K, double r, double T, double price, double sigma) {
    // Multiplying instead of using pow is generally faster
    double d1 = (log(S / K) + ( (r + (0.5 * sigma * sigma) ) * T ) ) / (sigma * sqrt(T));
    double d2 = d1 - ( sigma * sqrt(T) );
    double n1 = normalCDF(d1);
    double n2 = normalCDF(d2);
    double discount_factor = exp(-r * T);
    double call_price = (S * n1 ) - (K * discount_factor * n2);

    // std::cout << "d1: " << d1 << "\n";
    // std::cout << "d2: " << d2 << "\n";
    // std::cout << "n1: " << n1 << "\n";
    // std::cout << "n2: " << n2 << "\n";

    return call_price - price; // Function should be zero at the root, we are going to use Newton - Rhapsod iterative method
}

In [5]:
double S = 1169.5;    // Current stock price
double K = 1033.0;     // Strike price
double r = 0.9;    // Risk-free rate
double T = 1.0 * (30.0/360.0);   // Time to maturity
double price = 250.0;    // Observed option price
// For bisection method specifically
function_to_optimize(S, K, r, T, price, 0.001)

-38.859021

In [6]:
// Function to find implied volatility using the bisection method
std::vector<double> implied_volatility_bisection(double S, double K, double r, double T, double price, double lower_bound, double upper_bound, double tol = 1e-6, int max_iter = 1000) {
    // Define initial bounds for implied volatility
    // const double lower_bound = 0.001;
    // const double upper_bound = 2.0;
    double sigma_low = lower_bound;
    double sigma_high = upper_bound;
        

    // Ensure that observed price lies between calculated prices at initial bounds
    // We check for the "sign", the correct value is zero 0, so for bolzano one boundarie must be negative and
    // the other one positive, giving the multipication a negative number
    if (function_to_optimize(S, K, r, T, price, sigma_low) * function_to_optimize(S, K, r, T, price, sigma_high) > 0) {
        // std::cout << "OUT OF BOUNDS";
        return {NAN, NAN, NAN}; // Return NaN if bounds are incorrect
    }
    

    std::vector<double> result;
    // Bisection loop
    for (int iter = 0; iter < max_iter; ++iter) {
        double sigma_mid = (sigma_low + sigma_high) / 2;
        double call_price_mid = function_to_optimize(S, K, r, T, price, sigma_mid);
        // std::printf("The call price mid is  %.2f\n", call_price_mid);
        // std::cout << "The price mid is" << call_price_mid;

        // This call_price_mid here should ideally be equal to zero
        if (std::abs(call_price_mid) < tol) {
            result.push_back(sigma_mid);
            result.push_back(call_price_mid);
            result.push_back(iter);
            return result;
        }

        if (call_price_mid < 0) {
            sigma_low = sigma_mid;
        } else {
            sigma_high = sigma_mid;
        }
    }

    // std::cout << "No convergence";
    return {NAN, NAN, NAN}; // Return NaN if no convergence within max iterations
}

In [7]:
// double S = 1169.5;    // Current stock price
// double K = 1033.0;     // Strike price
// double r = 0.9;    // Risk-free rate
// double T = 1.0 * (30.0/360.0);   // Time to maturity
// double price = 250.0;    // Observed option price
// // For bisection method specifically
double lower_bound = 0.001;
double upper_bound = 2.0;
double tol = 1e-6;
double max_iter = 10000;

// Dato segunda fila. El T maturity lo saqué de python
// double S = 1211.5;    // Current stock price
// double K = 1033.0;     // Strike price
// double r = 0.9;    // Risk-free rate
// double T = 0.117421;   // Time to maturity
// double price = 270.602;    // Observed option price

// Dato tercera fila
double S = 1209.500	;    // Current stock price
double K = 1033.0;     // Strike price
double r = 0.9;    // Risk-free rate
double T = 0.117419	;   // Time to maturity
double price = 297.6530;    // Observed option price
	
implied_volatility_bisection(S, K, r, T, price, lower_bound, upper_bound, tol, max_iter)[0]

0.70161483

In [8]:
function_to_optimize(S, K, r, T, price, 2345)

911.84700

#### CALCULOS

In [9]:
// Function to adapt a 2D std::vector to an xt::xarray
// Funcion, de output un xarray. De parámetros input un vector de entrada
xt::xarray<double> adapt_vector_2d(const std::vector<std::vector<double>>& vec) {
    // Get the shape of the 2D vector
    std::vector<size_t> shape = {vec.size(), vec.empty() ? 0 : vec[0].size()};

    // Flatten the 2D vector into a 1D vector
    std::vector<double> flat_vec;
    for (const auto& row : vec) {
        // Simplemente insertamos cada vector de a uno en el vector nuevo
        flat_vec.insert(flat_vec.end(), row.begin(), row.end());
    }

    // Adapt the flattened vector as an xarray
    // We must traspose the final matrix to show correctly
    return xt::transpose( xt::adapt(flat_vec, shape) );
};

// CALCULOS
// TENEMOS QUE CALCULAR
// CHECK calcular promedios de opciones y stocks (avopt y avstock) momento a momento
// - desvíos estándar históricos (entre bid y price de opciones más fácil)
// - desvío estándar "by minute", hacer un shift primero de los datos o ver
// - time to maturities a todo momento

// Convertimos a segundos con la función ya definida para ser manipulado adecuadamente con xtensor
auto created_at_seconds = convert_dates_to_seconds(created_at);


xt::xarray<double> numeric_columns_xarray = adapt_vector_2d(numeric_columns);
// Vamos guardando para siempre el número de filas
int numberofrows = numeric_columns_xarray.shape()[0];
// Calculamos los avergae de opciones y de acciones(stock), y calculamos el std historical instantaneo
xt::xarray<double> avopt = xt::nanmean ( xt::view(numeric_columns_xarray, xt::all(),xt::range(0, 2)) , {1});
avopt.reshape({numberofrows,1}); // COL 4
xt::xarray<double> avstock = xt::nanmean ( xt::view(numeric_columns_xarray, xt::all(),xt::range(2, 4)) , {1});
avstock.reshape({numberofrows,1}); // COL 5
// Ojo el std dev, {1} es el eje, y el 1 dps es "sample standard deviation" (n-1) en vez de la population
xt::xarray<double> std_instant_historical = xt::stddev( xt::view( numeric_columns_xarray, xt::all(), xt::range(2,4) ) , {1}, 1) ;
std_instant_historical.reshape({numberofrows,1}); // COL 6
// Appendeamos al array original estos resultados como columnas
numeric_columns_xarray  = xt::concatenate(xt::xtuple(numeric_columns_xarray, avopt,avstock,std_instant_historical), 1);
std::cout << std_instant_historical;


// Para los otros los SHIFT vamos a hacer un stacking con una fila vacía
// y además seleccionamos HASTA la anteúltima fila, para eso contamos filas totales y restamos uno.
xt::xarray<double>empty_row {std::nan("")};
empty_row.reshape({1,1});
xt::xarray<double> under_Bid_prev = xt::concatenate(  xt::xtuple (  empty_row , xt::view( numeric_columns_xarray,  xt::range(0,numberofrows-1) , xt::keep(2) ) ),  0 ); // COL 7
xt::xarray<double> under_Ask_prev = xt::concatenate(  xt::xtuple (  empty_row , xt::view( numeric_columns_xarray,  xt::range(0, numberofrows-1 ), xt::keep(3) ) ),  0 ); // COL 8

// SEGUIMOS PREPARANDO PARA LOS STD BY MINUTE
// Concatenaos todas las columnas prmero
numeric_columns_xarray = xt::concatenate(xt::xtuple(numeric_columns_xarray, under_Bid_prev, under_Ask_prev), 1);
// Los average de los previous (los shifteados)
xt::xarray<double> avstockprev = xt::nanmean( xt::view (numeric_columns_xarray, xt::all(), xt::range(7,9) ), {1} );
avstockprev.reshape({numberofrows, 1}); 
// Reconcatenamos esta última fila con los average price de stock shifteados
numeric_columns_xarray  = xt::concatenate(xt::xtuple(numeric_columns_xarray, avstockprev), 1); // COL 9

// // Calculamos std by minute de una vez ahora entre los average de underbidask y uniderbidaskprev
// // que son avstock y avstockprev
// Creo un xarray temporal para esto porque no le gusta calcular cosas entre columnas NO contiguas lamentablemente
xt::xarray<double> avstockavprev = xt::view( numeric_columns_xarray, xt::all(), xt::keep(5,9) );
xt::xarray<double> std_by_minute = xt::stddev ( avstockavprev, {1}, 1); 
std_by_minute.reshape({numberofrows,1});
// Concatenamos al original
numeric_columns_xarray  = xt::concatenate(xt::xtuple(numeric_columns_xarray, std_by_minute), 1); // COL 10

// PASAMOS A LOS CALCULOS CON FECHAS
// Convertimos segundos created at a un xarray
xt::xarray<double> created_at_seconds_xarray = xt::adapt(created_at_seconds, {numberofrows,1});
// Apendeamos estos segundos
numeric_columns_xarray  = xt::concatenate(xt::xtuple(numeric_columns_xarray, created_at_seconds_xarray), 1); // COL 11
// Escribimos la fecha de vencimiento de opciones como vector 1D y la transformamos
std::vector<std::string> vencimiento = {{"10/20/2023 18:00"}};
auto vencimiento_seconds_str = convert_dates_to_seconds(vencimiento);
xt::xarray<double> vencimiento_seconds = xt::adapt (vencimiento_seconds_str, {1,1});
// Le restmoas entonces el tiempo a cada fila para obtener time to maturity
xt::xarray<double> timetomaturity_seconds =  vencimiento_seconds - created_at_seconds_xarray;
// Pasamos el tiempo a años!
xt::xarray<double> timetomaturity_years = timetomaturity_seconds / 31104000;
// Agregamos el timetomaturity_years al xarray
numeric_columns_xarray  = xt::concatenate(xt::xtuple(numeric_columns_xarray, timetomaturity_years), 1); // COL 12

// Comenzamos a calcular lo importante las volatilidades
// Definamos los parámetros de black-scholes
const double lower_bound = 0.001;
const double upper_bound = 2.0;
double K = 1033.0;  // Strike price, default 100
double r = 0.9;   // Risk-free interest rate, default 0.03
double tol = 1e-6;
int max_iter = 1000;

// Array vacío con zeros para guardar el resultado
xt::xarray<double> bisection_results = xt::zeros<double>({numberofrows});
// Usamos unsigned long long para for que es equivalente a un int positivo
for ( unsigned long long i = 0; i < numberofrows; ++i) {
    double S = numeric_columns_xarray(i, 5);  // avstock (column 5)
    double price = numeric_columns_xarray(i, 4); // avopt (column 4)
    double T = numeric_columns_xarray(i, 12); // time_to_maturity (column 12)
    bisection_results(i) = implied_volatility_bisection(S, K, r, T, price, lower_bound, upper_bound, tol, max_iter)[0];    
};
// Reshapeamos el resultado
bisection_results.reshape({numberofrows,1});
// Concantenamos
numeric_columns_xarray  = xt::concatenate(xt::xtuple(numeric_columns_xarray, bisection_results), 1); // COL 13

// // Extraemos sólo las columnas deseadas
auto selected_columns = xt::view(numeric_columns_xarray, xt::all(), xt::keep(6, 10, 13));
// selected_columns;

// // Exportamos a un csv
// std::ofstream file("selected_columns.csv");
std::ofstream file("cpp_selected_columns_exp_octubre.csv");
xt::dump_csv(file, selected_columns);
file.close();

numeric_columns_xarray
// YA ESTA TODO VIEJO CHECKEADO LOS NUMEROS Y EL CSV EXPORTADO,
// HACERLO PARA EL FULL Y PONERSE A SCRIPTEARLO EN C nomás!

{{ 1.308148},
 { 0.707107},
 { 2.12132 },
 ...,
 { 1.343503},
 { 0.707107},
 { 2.12132 }}

2.500000e+02,nan,1.211150e+03,⋯,1.694183e+09,1.174228e-01,nan
2.706020e+02,nan,1.211000e+03,⋯,1.694183e+09,1.174209e-01,nan
2.706520e+02,3.246540e+02,1.208000e+03,⋯,1.694183e+09,1.174190e-01,7.016150e-01
⋮,⋮,⋮,⋱,⋮,⋮,⋮
1.000010e+02,1.080000e+02,1.140100e+03,⋯,1.697740e+09,3.084491e-03,nan
1.000080e+02,1.080000e+02,1.143000e+03,⋯,1.697740e+09,3.076775e-03,nan
1.000080e+02,1.150000e+02,1.135000e+03,⋯,1.697741e+09,3.051698e-03,1.050902e+00


In [10]:
std::cout << xt::view(numeric_columns_xarray, xt::all(), xt::range(0,6))

{{  250.      ,          nan,  1211.15    ,  1213.      ,   250.      ,
   1212.075   },
 {  270.602   ,          nan,  1211.      ,  1212.      ,   270.602   ,
   1211.5     },
 {  270.652   ,   324.654   ,  1208.      ,  1211.      ,   297.653   ,
   1209.5     },
 ...,
 {  100.001   ,   108.      ,  1140.1     ,  1142.      ,   104.0005  ,
   1141.05    },
 {  100.008   ,   108.      ,  1143.      ,  1144.      ,   104.004   ,
   1143.5     },
 {  100.008   ,   115.      ,  1135.      ,  1138.      ,   107.504   ,
   1136.5     }}

@0x7f68f623cce0

In [11]:
std::cout << numeric_columns_xarray;

{{ 2.500000e+02,           nan,  1.211150e+03, ...,  1.694183e+09,  1.174228e-01,
            nan},
 { 2.706020e+02,           nan,  1.211000e+03, ...,  1.694183e+09,  1.174209e-01,
            nan},
 { 2.706520e+02,  3.246540e+02,  1.208000e+03, ...,  1.694183e+09,  1.174190e-01,
   7.016150e-01},
 ...,
 { 1.000010e+02,  1.080000e+02,  1.140100e+03, ...,  1.697740e+09,  3.084491e-03,
            nan},
 { 1.000080e+02,  1.080000e+02,  1.143000e+03, ...,  1.697740e+09,  3.076775e-03,
            nan},
 { 1.000080e+02,  1.150000e+02,  1.135000e+03, ...,  1.697741e+09,  3.051698e-03,
   1.050902e+00}}